In [16]:
import math
import logging
from typing import Union
from urllib.parse import urlparse, parse_qsl, urlsplit

import pandas as pandas
import seaborn as sb
import json
import requests
import fhirclient

from IPython.display import display, HTML
#fhir server clients and configuration
from fhirclient import client
import fhirclient.models.patient as p
import fhirclient.models.observation as o
import fhirclient.models.bundle as b


In [17]:
#Constants
HAPI_URL = 'http://hapi.fhir.org/baseR4'

In [18]:
#Configure settings
logging.basicConfig(filename="newfile.log", 
                    format='%(asctime)s %(message)s', 
                    filemode='w')
logger=logging.getLogger()
logger.setLevel(logging.DEBUG)

In [19]:
#Test library on HAPI-FHIR test server
resp = requests.get(HAPI_URL)
print(resp.json())

settings = {
    'app_id': 'fhir',
    'api_base': 'http://hapi.fhir.org/baseR4/'
}
#Test sever connection
SMART.prepare()  # prints True after fet ching CapabilityStatement

#test a patient query
SMART = client.FHIRClient(settings=settings)    
patient = p.Patient.read('697505', SMART.server)
p_name = SMART.human_name(patient.name[0])
# 'Dorothy Mayer'
print('Patient Name:', p_name)

{'resourceType': 'OperationOutcome', 'text': {'status': 'generated', 'div': '<div xmlns="http://www.w3.org/1999/xhtml"><h1>Operation Outcome</h1><table border="0"><tr><td style="font-weight: bold;">ERROR</td><td>[]</td><td><pre>This is the base URL of FHIR server. Unable to handle this request, as it does not contain a resource type or operation name.</pre></td>\n\t\t\t\t\t\n\t\t\t\t\n\t\t\t</tr>\n\t\t</table>\n\t</div>'}, 'issue': [{'severity': 'error', 'code': 'processing', 'diagnostics': 'This is the base URL of FHIR server. Unable to handle this request, as it does not contain a resource type or operation name.'}]}
Patient Name: Dorothy Mayer


## Search for observation in FHIR-SERVER
Observation    
85354-9: LOINC CODE, Blood pressure panel with all children optional.
Will look for observations of blood preassure 


In [20]:

def search_observation(obs_code: str, server) -> pandas.DataFrame:
    """ 
        Recibes a code conforming to SNOMED-CT 
        e.j. http://bioportal.bioontology.org/ontologies/SNOMEDCT/?p=classes&conceptid=http%3A%2F%2Fpurl.bioontology.org%2Fontology%2FSNOMEDCT%2F38341003&jump_to_nav=true

        Args:
            obs_code [str] -- SNOMED-CT or LOINC conforming code
            server   [   ] -- instance of fhirclient server
        
        Returns:
            iatros_df [pandas.DataFrame]
    """
    # Create search query
    fs = o.Observation.where(struct = {'status':'final',
                                    'code':{'$and': [obs_code]}
                                        }
                            )
    # Perfom query to receive a Bundle resourceType since it contains pagination link.
    bundle = fs.perform(server)
    #print(json.dumps(bundle.as_json(), indent=2))
    
    # Pass bundle to handel pagination and save entries in Dataframe
    iatros_df = handle_pagination(bundle)
    
    return iatros_df

In [ ]:
def handle_pagination(bundle: fhirclient.models.bundle) -> pd.DataFrame:
    """
        Handles a resourceType: "Bundle" entries, if pagination
        link avalable iterates looking for next page entries and
        queries the server for the next page.

        Arguments:
            bundle -- fhirclient.models.bundle

        Returns:
            pandas.DataFrame -- A Data frame containing inputs from
                                 the paginated  requests.
    """
    frames = []
    while True:
        entries = [be.resource for be in bundle.entry] if bundle is not None and bundle.entry is not None else None
        
        print('Retrieved {}/{} entries...'.format(len(bundle.entry) if      
        
        bundle.entry else 0, bundle.total if bundle.total else ' '))
        #Get a temp dataframe from current bundle entries
        temp_df =  append_entries_to_dataset(bundle)
        frames.append(temp_df)
        #Look for a pagination link
        if entries is not None and len(entries) > 0:
            next_link = get_next_link_in(bundle)  
            url_params = get_next_url_params(next_link)
            if len(url_params) > 1:
                #Query for next page
                response = requests.get(HAPI_URL, params=url_params)
                if response.status_code == 200:
                    #Initialize a bundle object from request response as json
                    try: 
                        bundle = b.Bundle(response.json())
                    except Exception as e:
                        print('An error ocurred while creating Bundle object')
                        print(e)
                        print('Error query: ', next_link)
                        return pd.concat(frames)
        else:
             return pd.concat(frames)

In [21]:
#Reuse SMART server instance
iatros_df = search_observation('85354-9', SMART.server)
iatros_df.reset_index(drop=True, inplace=True)
display(HTML(iatros_df.to_html()))


NameError: name 'handle_pagination' is not defined